In [1]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import *

# Create a test environment for use later
board_size = 9
environment = make("halite", configuration={"size": board_size, "startingHalite": 1000})
agent_count = 4

In [2]:
import random
from board_viz import draw_game, draw_board

def vector_distance(origin: Point, dest: Point):
    center = Point(board_size // 2, board_size // 2)
    return (dest - origin + center) % board_size - center

def manhattan_distance(p1: Point, p2: Point):
    dp = abs(vector_distance(p1, p2))
    return dp.x + dp.y

def magnitude(p):
    return abs(p.x) + abs(p.y)

def safe_float_div(p: Point, scalar):
    ''' floating point divide by a scalar number, returns zeros instead raising an exception'''
    return Point(p.x/scalar if p.x != 0 else 0, p.y/scalar if p.y != 0 else 0)

def direction(origin, destination):
    ''' Returns a unit vector in the direction from origin to destination'''
    return safe_float_div(vector_distance(origin, destination), manhattan_distance(origin, destination))

def halite_move_force(ship, cell):
    ''' calculates the pull force vector of halite on the ship'''
    dist = manhattan_distance(ship.position, cell.position)
    # divide halite by distance to get initial magnitude
    magnitude = cell.halite / dist if dist > 0 else 0
    return direction(ship.position, cell.position) * magnitude

stay_power = 1.4
cluster_power = 0.1

def force_to_action(best_move_force, sum_move_force, stay_force):
    ''' Turns three input values into a Ship action
    best_move_force is the best calculated single cell to move to
    sum_move_force is the best overall direction to move toward
    stay_force is the magnitude of how good our current cell is
    we use stay_power and cluster_power to scale them
    '''
    move_force = best_move_force + sum_move_force * cluster_power
    stay_force = stay_force * stay_power
    if stay_force > abs(move_force.x) and stay_force > abs(move_force.y):
        return None
    if abs(move_force.x) > abs(move_force.y):
        if move_force.x > 0:
            return ShipAction.EAST
        return ShipAction.WEST
    if move_force.y > 0:
        return ShipAction.NORTH
    return ShipAction.SOUTH

def next_move(board, ship, t):
    if t == 0:
        return ShipAction.CONVERT
    sum_move_force = Point(0, 0)
    best_move_force = Point(0, 0)
    for cpos, cell in board.cells.items():
        move_force = halite_move_force(ship, cell)
        best_move_force = max(move_force, best_move_force, key=magnitude)
        sum_move_force += move_force
    return force_to_action(best_move_force, sum_move_force, board.cells[ship.position].halite)

environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)
t = 0
def step(board, t):
    for ship in board.ships.values():
        ship.next_action = next_move(board, ship, t)
    for shipyard in board.shipyards.values():
        if len(board.ships) < 2:
            shipyard.next_action = ShipyardAction.SPAWN

boards = [board]
for i in range(50):
    step(board, t)
    t += 1
    board = board.next()
    boards.append(board)
    if len(board.ships)+ len(board.shipyards)==0:
        break
draw_game(boards)


interactive(children=(IntSlider(value=0, description='t', max=49), Checkbox(value=True, description='cell_hali…

In [21]:
import time

def score_solution(board, positions):
    score = 0.0
    halite = {pos : cell.halite for pos, cell in board.cells.items()}
    prev_pos = None
    for pos in positions:
        if prev_pos:
            if pos == prev_pos:
                score += halite[pos] * 0.25
                halite[pos] *= 0.75
        prev_pos = pos
    return score

def is_valid(board, t, t_max, positions, candidate, origin, destination):
    # assume candidate is 1 away from position[t-1]
    # assume that position[0] is 1 away origin
    if candidate == origin:
        return False
    if candidate == destination:
        return False
    if manhattan_distance(candidate, destination) > t_max - t:
        return False
    return True

def find_candidates(board, t, t_max, positions, origin, destination):
    pos = positions[t - 1] if t > 0 else origin
    legal_moves = [Point(1,0), Point(-1, 0), Point(0,1), Point(0,-1), Point(0,0)]
    candidates = [(m + pos) % board.configuration.size for m in legal_moves] 
    return [candidate for candidate in candidates if is_valid(board, t, t_max, positions, candidate, origin, destination)]

def dfs_solutions(board, t, t_max, positions, origin, destination):
    best_solution = [], 0
    if t == t_max:
        score = score_solution(board, positions)
        return positions, score
    candidates = find_candidates(board, t, t_max, positions, origin, destination)
    for candidate in candidates:
        positions[t] = candidate
        solution, score = dfs_solutions(board, t + 1, t_max, positions, origin, destination)
        if score > best_solution[1]:
            best_solution = solution[:], score
    return best_solution    

board_size = 8
environment = make("halite", configuration={"size": board_size, "startingHalite": 1000})
agent_count = 1
environment.reset(agent_count)
state = environment.state[0]
board = Board(state.observation, environment.configuration)

boards = [board]
ship = list(board.ships.values())[0]
start_pos = ship.position

start_time = time.time()
solution = dfs_solutions(board, 0, 8, [None]*8, start_pos, start_pos)
print(solution)
print('time:', time.time()-start_time, 'seconds')

action_lookup = {
    Point(0,0): None,
    Point(1,0): ShipAction.EAST,
    Point(-1,0): ShipAction.WEST,
    Point(0,1): ShipAction.NORTH,
    Point(0,-1): ShipAction.SOUTH
}
ship.next_action = ShipAction.CONVERT
board = board.next()
boards.append(board)
shipyard = list(board.shipyards.values())[0]
shipyard.next_action = ShipyardAction.SPAWN
board = board.next()
boards.append(board)
for pos in solution[0] +[start_pos]:
    ship = list(board.ships.values())[0]
    ship.next_action = action_lookup[pos - ship.position]
    board = board.next()
    boards.append(board)
boards.append(board)
draw_game(boards)



([(3, 3), (3, 2), (3, 2), (3, 2), (4, 2), (4, 2), (4, 2), (4, 2)], 193.984375)
time: 5.993701219558716 seconds


interactive(children=(IntSlider(value=0, description='t', max=11), Checkbox(value=True, description='cell_hali…